In [2]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

Getting the url

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result=requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Sun, 06 Dec 2020 21:26:47 GMT', 'Server': 'mw1332.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'X-Request-Id': 'b6c401b3-162f-4676-bef4-faa0d717b869', 'Last-Modified': 'Sun, 06 Dec 2020 20:02:49 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '47705', 'X-Cache': 'cp1081 miss, cp1089 hit/27', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Report-To': '{ "group": "wm_nel", "max_age": 86400, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }', 'NEL': '{ "report_to": "wm_nel", "max_age": 8640

Getting data from URL and cleaning it

In [5]:
soup=BeautifulSoup(result.content,'html.parser')
table=soup.find('table')
trs=table.find_all('tr')
rows=[]
for tr in trs:
    i=tr.find_all('td')
    if(i):
        rows.append(i)
lst=[]
for row in rows:
    postalcode=row[0].text.rstrip()
    borough=row[1].text.rstrip()
    neighbourhood=row[2].text.rstrip()
    if(borough!='Not Assigned'):
        if(neighbourhood=='Not Assigned'):
            neighbourhood=borough
        lst.append([postalcode,borough,neighbourhood])
lst

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M

Convert the above array into a dataframe

In [6]:
cols=['PostalCode','Borough','Neighbourhood']
df=pd.DataFrame(lst,columns=cols)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
print(df.shape)

(180, 3)


In [28]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighbourhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
dfgeo= pd.read_csv(r'http://cocl.us/Geospatial_data')
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
dfgeo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the two dataframes df and dfgeo

In [8]:
df2=pd.merge(df,dfgeo,on="PostalCode",how="left")

In [9]:
df2.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN
1,M2A,Not assigned,Not assigned,NaN,NaN
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
